### 读取数据库中的所有表名

In [1]:
import pymysql.cursors 
 
 
dbname = 'db_eastmoney'
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='0000',
                             db=dbname,
                             charset='utf8',
                             cursorclass=pymysql.cursors.DictCursor)
results =[]
try:
    with connection.cursor() as cursor:
        sql = '''SHOW TABLES'''
        cursor.execute(sql)
        result = cursor.fetchall() 
        for i in range(len(result)):
            results.append(result[i]['Tables_in_'+dbname])
finally:
    connection.close()

print(len(results),results)

60 ['czce_ap', 'czce_cf', 'czce_cj', 'czce_cy', 'czce_fg', 'czce_jr', 'czce_lr', 'czce_ma', 'czce_oi', 'czce_pf', 'czce_pk', 'czce_pm', 'czce_ri', 'czce_rm', 'czce_rs', 'czce_sa', 'czce_sf', 'czce_sm', 'czce_sr', 'czce_ta', 'czce_ur', 'czce_wh', 'czce_zc', 'dce_a', 'dce_b', 'dce_bb', 'dce_c', 'dce_cs', 'dce_eb', 'dce_eg', 'dce_fb', 'dce_i', 'dce_j', 'dce_jd', 'dce_jm', 'dce_l', 'dce_lh', 'dce_m', 'dce_p', 'dce_pg', 'dce_pp', 'dce_rr', 'dce_v', 'dce_y', 'shfe_ag', 'shfe_al', 'shfe_au', 'shfe_bu', 'shfe_cu', 'shfe_fu', 'shfe_hc', 'shfe_ni', 'shfe_pb', 'shfe_rb', 'shfe_ru', 'shfe_sn', 'shfe_sp', 'shfe_ss', 'shfe_wr', 'shfe_zn']


czce_ta


In [3]:
# 写入保证金和合约单位相关数据
import pandas as pd
df = pd.read_excel(r'C:\Users\17218\Desktop\各期货品种交易规则简表20210525.xlsx',sheet_name='自建不重复')#,usecols=[0，1，3])
df = df[['代码','合约单位2','保证金率2']]
code_dic = dict(zip(df['代码'].tolist(),list(zip(df['合约单位2'].tolist(),df['保证金率2'].tolist()))))

{'fu': (10, 0.14),
 'ag': (15, 0.17),
 'al': (5, 0.13),
 'au': (1000, 0.12),
 'cu': (5, 0.14),
 'pb': (5, 0.13),
 'rb': (10, 0.14),
 'ru': (10, 0.14),
 'wr': (10, 0.4),
 'ni': (1, 0.14),
 'sn': (1, 0.14),
 'zn': (5, 0.13),
 'ss': (5, 0.12),
 'bu': (10, 0.15),
 'sp': (10, 0.12),
 'hc': (10, 0.14),
 'a': (10, 0.16),
 'b': (10, 0.13),
 'c': (10, 0.14),
 'cs': (10, 0.11),
 'j': (100, 0.15),
 'l': (5, 0.12),
 'm': (10, 0.12),
 'p': (10, 0.14),
 'v': (5, 0.12),
 'jm': (60, 0.15),
 'y': (10, 0.12),
 'rr': (10, 0.1),
 'i': (100, 0.16),
 'pp': (5, 0.12),
 'fb': (10, 0.15),
 'bb': (500, 0.5),
 'jd': (5, 0.13),
 'lh': (16, 0.19),
 'eb': (5, 0.16),
 'eg': (10, 0.15),
 'pg': (20, 0.16),
 'PM': (50, 0.4),
 'RI': (20, 0.4),
 'WH': (20, 0.4),
 'TA': (5, 0.1),
 'CF': (5, 0.11),
 'OI': (10, 0.12),
 'SR': (10, 0.11),
 'FG': (20, 0.14),
 'RS': (10, 0.4),
 'RM': (10, 0.12),
 'MA': (10, 0.12),
 'JR': (20, 0.4),
 'SF': (5, 0.11),
 'SM': (5, 0.11),
 'CY': (5, 0.11),
 'AP': (10, 0.12),
 'UR': (20, 0.11),
 'CJ'

In [11]:
import pymysql
import pandas as pd
import re
import itertools
 
abitrage_dic = {}
conn = pymysql.connect(host = "localhost",port=3306,user='root',passwd='0000',db='db_eastmoney',charset="utf8")
for i in results:
    sql_query = 'SELECT * FROM '+i
    # 从mysql中读取表
    df1 = pd.read_sql(sql_query, con=conn)
    # 中文转码
    df1.name = df1.name.apply(lambda x:eval(repr(x).replace('\\\\', '\\')))
    # TA特殊处理
    if 'TA' in df1.loc[0,'name']:
        # 提取字母
        name = ''.join(re.findall(r'[A-Za-z]', df1.loc[0,'name']))   
    else:
        # 提取中文
        name = ''.join(re.findall(u"[\u4e00-\u9fa5]+", df1.loc[0,'name']))
    # 提取字母  code  AP 
    code = ''.join(re.findall(r'[A-Za-z]', df1.loc[0,'code']))
    # 去掉主力和次主力的行 带有- 的行
    df1 = df1[(df1.name.str.contains('主力')==False) & (df1.new_p!='-')]
    # 转换类型
    df1['new_p'] = df1['new_p'].astype('float64')
    df1['volume'] = df1['volume'].astype('int64')
    df1['money'] = df1['new_p']*df1['volume']
    
    if code in code_dic.keys():
        df1['bzjl'] = code_dic[code][1]   # 保证金率
        df1['hydw'] = code_dic[code][0]   # 合约单位
    df1 = df1[df1['money']>1000*10000]    # 成交额 > 2000万
    
    df1['bzj'] = df1['bzjl']* df1['hydw']*df1['new_p']
    df1 = df1[df1['bzj']<10000]    # 保证金小于10000
    if len(df1)>2:
        cate_list = df1.code.tolist()
        abitrage_dic[name] = list(itertools.combinations(cate_list, 2))
        print(name,(min(df1['bzj'].tolist()),max(df1['bzj'].tolist())),'\n',list(itertools.combinations(cate_list, 2)))
conn.close()  #使用完后记得关掉
# print(abitrage_dic)

苹果 (7353.599999999999, 7741.2) 
 [('AP110', 'AP111'), ('AP110', 'AP201'), ('AP110', 'AP205'), ('AP111', 'AP201'), ('AP111', 'AP205'), ('AP201', 'AP205')]
一号棉花 (8530.5, 8714.75) 
 [('CF109', 'CF111'), ('CF109', 'CF201'), ('CF111', 'CF201')]
甲醇 (2959.2, 3058.7999999999997) 
 [('MA108', 'MA109'), ('MA108', 'MA110'), ('MA108', 'MA201'), ('MA109', 'MA110'), ('MA109', 'MA201'), ('MA110', 'MA201')]
菜粕 (3184.7999999999997, 3408.0) 
 [('RM108', 'RM109'), ('RM108', 'RM111'), ('RM108', 'RM201'), ('RM109', 'RM111'), ('RM109', 'RM201'), ('RM111', 'RM201')]
纯碱 (5714.8, 6169.8) 
 [('SA108', 'SA109'), ('SA108', 'SA201'), ('SA109', 'SA201')]
硅铁 (4515.5, 4565.0) 
 [('SF107', 'SF108'), ('SF107', 'SF109'), ('SF107', 'SF110'), ('SF107', 'SF111'), ('SF107', 'SF201'), ('SF108', 'SF109'), ('SF108', 'SF110'), ('SF108', 'SF111'), ('SF108', 'SF201'), ('SF109', 'SF110'), ('SF109', 'SF111'), ('SF109', 'SF201'), ('SF110', 'SF111'), ('SF110', 'SF201'), ('SF111', 'SF201')]
锰硅 (3985.3, 4057.9000000000005) 
 [('SM107',

In [4]:
df = pd.DataFrame()
import pandas as pd
for key,value in abitrage_dic.items():
    single_dic = {"品种":key,"近月":[x[0] for x in value],"远月":[x[1] for x in value]}
    single_df = pd.DataFrame(single_dic)
    df = pd.concat([df,single_df],axis=0)
df.reset_index(inplace=True,drop=True)
df.to_csv("套利品种.csv",index=False)
df

,品种,近月,远月
0,苹果,AP110,AP111
1,苹果,AP110,AP112
2,苹果,AP110,AP201
3,苹果,AP110,AP205
4,苹果,AP111,AP112
...,...,...,...
190,不锈钢,ss2107,ss2109
191,不锈钢,ss2107,ss2110
192,不锈钢,ss2108,ss2109
193,不锈钢,ss2108,ss2110


In [5]:
df = pd.read_csv("套利品种.csv")
df

,品种,近月,远月
0,苹果,AP110,AP111
1,苹果,AP110,AP112
2,苹果,AP110,AP201
3,苹果,AP110,AP205
4,苹果,AP111,AP112
...,...,...,...
190,不锈钢,ss2107,ss2109
191,不锈钢,ss2107,ss2110
192,不锈钢,ss2108,ss2109
193,不锈钢,ss2108,ss2110
